In [1]:
import pandas as pd
from MKtest import *
import numpy as np

In [2]:
d = dict(A=1, B=2, C=3, D=4)

In [3]:
file = "/home/sur/micropopgen/exp/today/mktest/Streptococcus_sp_57750/mk_results.Buccal.mucosa_Tongue.dorsum.txt"
pd.read_csv(file, sep="\t").head()

,gene,contig,start,end,Dn,Ds,Pn,Ps,hg,hg.pval
0,1156433.3.peg.1,CP006776,1134,2262,6,3,29,71,0.238095,0.037896
1,1156433.3.peg.2,CP006776,2271,2427,0,0,2,15,0.187500,0.352381
2,1156433.3.peg.3,CP006776,2531,2897,0,0,3,13,0.285714,0.447368
3,1156433.3.peg.4,CP006776,2935,4101,1,1,3,1,2.000000,1.000000
4,1156433.3.peg.5,CP006776,4162,4562,2,0,13,10,0.424242,0.622079


In [4]:
class ARGS:
    def __init__(self, indir, metadata_file,
                group1, group2, test='hg',
                outfile='mk_results', min_count=1,
                nrows=float('inf'), tables='mk_tables',
                pseudocount=0, permutations=0, seed=5):
        
        self.indir = indir
        self.metadata_file = metadata_file
        self.group1 = group1
        self.group2 = group2
        self.test = test
        self.outfile = outfile
        self.min_count = min_count
        self.nrows = nrows
        self.tables = tables
        self.pseudocount = pseudocount
        self.permutations = permutations
        self.seed = seed


In [22]:
args = ARGS(indir='/home/sur/micropopgen/data/test_data/midas/merged.snps//Streptococcus_sp_57750/',
            metadata_file='/home/sur/micropopgen/data/test_data/midas/map.txt',
            group1='Buccal mucosa',
            group2='Tongue dorsum',
            outfile='mktest/Streptococcus_sp_57750//mk_results.Buccal.mucosa_Tongue.dorsum.txt',
            tables='mktest/Streptococcus_sp_57750//mk_tables.Buccal.mucosa_Tongue.dorsum.txt',
            nrows=2000,
            pseudocount=1,
            permutations=10,
            seed=5,
            test='all')

In [23]:
print("Checking MIDAS files exist")
confirm_midas_merge_files(args)

# Read mapping files
# Create dictionaries that have all the samples per group (Groups),
# and the group to which each sample belongs (Samples)
# Probably should change this to pandas
print("Read metadata")
Samples, Groups = process_metadata_file(args.metadata_file)

print("Calculate MK contingency tables")
MK, Genes = calculate_contingency_tables(Samples, Groups, args)
# print(MK)
# print([MK])


Checking MIDAS files exist
	All files found
Read metadata
Calculate MK contingency tables
	Read snps_info.txt
============HEADERs============
>Site id: site_id
>Contig: ref_id
>Position: ref_pos
>Ref allele: ref_allele
>Major allele: major_allele
>Minor allele: minor_allele
>Locus type: locus_type
>Gene id: gene_id
>Aminoacids: amino_acids

Number of sites: 1757
Number of genes: 20
	Chose sites based on depth in groups to compare
Number of sites: 768
Number of genes: 20
Sites with counts: 768
	Read frequencies and calculate
Processing snp_freq.txt
Number of sites: 768
Number of genes: 20
Sites with counts: 768
Genes with MK: 16


In [24]:
if args.permutations > 0:
        MK = [MK]
        print("Permuting")
        print("Seed is {}".format(str(args.seed)))
        np.random.seed(args.seed)
        for i in range(args.permutations):
            Sp, Gp = process_metadata_file(args.metadata_file, permute=True)
            mk, genes = calculate_contingency_tables(Sp, Gp, args)
            MK.append(mk)

            print("========")
            # print(MK)
            print(len(MK))

print(MK)

Permuting
Seed is 5
	Read snps_info.txt
============HEADERs============
>Site id: site_id
>Contig: ref_id
>Position: ref_pos
>Ref allele: ref_allele
>Major allele: major_allele
>Minor allele: minor_allele
>Locus type: locus_type
>Gene id: gene_id
>Aminoacids: amino_acids

Number of sites: 1757
Number of genes: 20
	Chose sites based on depth in groups to compare
Number of sites: 1609
Number of genes: 20
Sites with counts: 1609
	Read frequencies and calculate
Processing snp_freq.txt
Number of sites: 1609
Number of genes: 20
Sites with counts: 1609
Genes with MK: 18
2
	Read snps_info.txt
============HEADERs============
>Site id: site_id
>Contig: ref_id
>Position: ref_pos
>Ref allele: ref_allele
>Major allele: major_allele
>Minor allele: minor_allele
>Locus type: locus_type
>Gene id: gene_id
>Aminoacids: amino_acids

Number of sites: 1757
Number of genes: 20
	Chose sites based on depth in groups to compare
Number of sites: 1638
Number of genes: 20
Sites with counts: 1638
	Read frequencies 

In [29]:
# Get list of tests to perform
test = args.test
supported_tests = ['NI', 'ratio', 'hg',
                   'G', 'G_Yates', 'G_Williams',
                   'alpha']
if test == 'all':
    test = supported_tests
elif test not in supported_tests:
    raise ValueError("Test not supported")
else:
    test = [test]

# Create header
header_base = ['gene', 'contig', 'start', 'end',
               'Dn', 'Ds', 'Pn', 'Ps']
pval_list = [''.join([t, '.pval']) for t in test]
header = header_base + test + pval_list
print(test)
print(pval_list)


['NI', 'ratio', 'hg', 'G', 'G_Yates', 'G_Williams', 'alpha']
['NI.pval', 'ratio.pval', 'hg.pval', 'G.pval', 'G_Yates.pval', 'G_Williams.pval', 'alpha.pval']


In [30]:
MK_true = MK[0]
MK_true
gene = list(MK_true.keys())[0]
mk = MK[0][gene]
print(gene)
print(mk)

1156433.3.peg.1


In [31]:
perm_table = np.full(shape=(int(args.permutations + 1), len(test)), fill_value=np.nan)
row = 0
for p in MK:
    print("=========")
    # print(p)
    if gene in p:
        p_stat = calculate_statistic(p[gene], test, pseudocount=args.pseudocount)
        p_res = [p_stat[t] for t in test]
        perm_table[row] = p_res
    
    row = row + 1
perm_table

perm_pvals <- (perm_table >= perm_table[0]).sum(axis=0) / (args.permutations + 1)

array([[0.62324929, 4.2       , 0.23809524, 4.90696487, 3.58029174,
        4.65462305, 0.76190476],
       [0.31875876, 2.08333333, 0.48      , 0.25854096, 0.05807321,
        0.19961951, 0.52      ],
       [0.31875876, 2.08333333, 0.48      , 0.25854096, 0.05807321,
        0.19961951, 0.52      ],
       [0.31875876, 2.08333333, 0.48      , 0.25854096, 0.05807321,
        0.19961951, 0.52      ],
       [0.31875876, 2.08333333, 0.48      , 0.25854096, 0.05807321,
        0.19961951, 0.52      ],
       [0.31875876, 2.08333333, 0.48      , 0.25854096, 0.05807321,
        0.19961951, 0.52      ],
       [0.31875876, 2.08333333, 0.48      , 0.25854096, 0.05807321,
        0.19961951, 0.52      ],
       [0.31875876, 2.08333333, 0.48      , 0.25854096, 0.05807321,
        0.19961951, 0.52      ],
       [0.31875876, 2.08333333, 0.48      , 0.25854096, 0.05807321,
        0.19961951, 0.52      ],
       [0.31875876, 2.08333333, 0.48      , 0.25854096, 0.05807321,
        0.19961951, 0.5

In [ ]:
MK[0][gene].Ds

In [ ]:
p_stat

In [33]:
perm_table

array([[0.62324929, 4.2       , 0.23809524, 4.90696487, 3.58029174,
        4.65462305, 0.76190476],
       [0.31875876, 2.08333333, 0.48      , 0.25854096, 0.05807321,
        0.19961951, 0.52      ],
       [0.31875876, 2.08333333, 0.48      , 0.25854096, 0.05807321,
        0.19961951, 0.52      ],
       [0.31875876, 2.08333333, 0.48      , 0.25854096, 0.05807321,
        0.19961951, 0.52      ],
       [0.31875876, 2.08333333, 0.48      , 0.25854096, 0.05807321,
        0.19961951, 0.52      ],
       [0.31875876, 2.08333333, 0.48      , 0.25854096, 0.05807321,
        0.19961951, 0.52      ],
       [0.31875876, 2.08333333, 0.48      , 0.25854096, 0.05807321,
        0.19961951, 0.52      ],
       [0.31875876, 2.08333333, 0.48      , 0.25854096, 0.05807321,
        0.19961951, 0.52      ],
       [0.31875876, 2.08333333, 0.48      , 0.25854096, 0.05807321,
        0.19961951, 0.52      ],
       [0.31875876, 2.08333333, 0.48      , 0.25854096, 0.05807321,
        0.19961951, 0.5

In [37]:
perm_table[:,0]

array([0.62324929, 0.31875876, 0.31875876, 0.31875876, 0.31875876,
       0.31875876, 0.31875876, 0.31875876, 0.31875876, 0.31875876,
       0.31875876])

In [47]:
sum(perm_table[:,0] >= perm_table[:,0][0]) / (args.permutations + 1)

0.09090909090909091

In [60]:
perm_table >= perm_table[0]

array([[ True,  True,  True,  True,  True,  True,  True],
       [False, False,  True, False, False, False, False],
       [False, False,  True, False, False, False, False],
       [False, False,  True, False, False, False, False],
       [False, False,  True, False, False, False, False],
       [False, False,  True, False, False, False, False],
       [False, False,  True, False, False, False, False],
       [False, False,  True, False, False, False, False],
       [False, False,  True, False, False, False, False],
       [False, False,  True, False, False, False, False],
       [False, False,  True, False, False, False, False]])

array([0.09090909, 0.09090909, 1.        , 0.09090909, 0.09090909,
       0.09090909, 0.09090909])

In [63]:
perm_table[:,1]

array([4.2       , 2.08333333, 2.08333333, 2.08333333, 2.08333333,
       2.08333333, 2.08333333, 2.08333333, 2.08333333, 2.08333333,
       2.08333333])

In [43]:
MK[0][gene].Ds

3

In [ ]:
[p_stat[t] for t in test]

In [ ]:
p[gene].Ps

In [ ]:
perm_table = np.full(shape=(11,3), fill_value=np.nan)
print(perm_table)
[1,2,3]

In [ ]:
perm_table[0] = [1,2,3]

In [ ]:
perm_table

In [ ]:
p_stat

In [ ]:
np.nan((int(args.permutations + 1), len(test)))

In [ ]:
len(test)

In [ ]:
np.ndarray(shape=(int(args.permutations + 1), len(test)), buffer=None)

In [ ]:
np.ndarray?

In [ ]:
np.empty((1,2))

In [ ]:
np.empty?

In [ ]:
np.array()